## 로라 병합 및 허깅페이스 업로드하기
이제 학습이 끝났으니 해당 모델을 허깅페이스에 업로드 해보도록 하겠습니다.


###주의 사항
이번 실습을 진행하기 위해서는 학습한 모델이 현재 경로에 존재해야 하고, 또한 기존에 실습했던 주피터
노트북을 종료하고 새로운 주피터 노트북을 생성해야 합니다.(런팟을 종료하는 것이 아닙니다.) 아래의
주의 사항을 따르면 됩니다.
1. 해당 실습은 이전 실습과 이어서 진행한다고 가정합니다. 따라서 현재 실습 환경에는 학습이 현재
경로의 qwen2-7b-rag-ko 내부에 checkpoint-285 디렉토리가 생성된 상태여야 합니다.
2. 이전에 학습을 진행했던 주피터 노트북 상단의 Kernel 버튼을 클릭하고 이어서 ShudDown
Kernel 버튼을 클릭한 후에 이번 실습을 진행해야 합니다. 해당 주피터 노트북을 종료하는 과
정으로 커널을 종료한다고 표현합니다. 저자의 경우 모델을 학습했던 주피터 노트북 파일이 ‘1.
tuning_example.ipynb‘ 이므로 해당 주피터 노트북의 커널을 종료하였습니다.
3. 이제 주피터 노트북을 새로 생성하세요. 저자의 경우 ‘2. merge and upload.ipynb‘라는 이름의 주
피터 노트북을 생성하였습니다. 그리고 이제 이 노트북에서 실습을 진행하겠습니다.
4. 참고로 런팟에서 주피터 노트북으로 실습할 때 에러가 나는 경우는 대부분 해당 주피터 노트북의
Kernel 버튼을 클릭하고 이어서 Restart Kernel을 하고 나서 pip 부분은 건너뛰고 다시
실습하면 해결되는 경우가 많으므로 참고하세요.


### 1. LoRA병합
학습이 완료된 후에는 LoRA 어댑터를 베이스 모델과 병합하여 독립적인 모델로 만들 수 있습니다. 먼저 필요한 라이브러리들을 임포트하고 경로를 설정합니다.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 경로 설정
base_model_path = "Qwen/Qwen2-7B-Instruct"
adapter_path = "./qwen2-7b-rag-ko/checkpoint-285"
merged_model_path = "./output_dir"

# 디바이스 설정
device_arg = {"device_map": "auto"}

- adapter_path는 학습된 LoRA어댑처가 저장된 경로입니다. 여기서 qwen2-7b-rag-ko
안의 checkpoint-285를 사용합니다.
- base_model_id는 원본 베이스 모델입니다. LoRA 학습 시 사용했던 것과 동일한 모델이어야 합니다
- merged_path는 병합된 모델이 저장될 경로입니다. 여기서는 “merged” 라는 폴더에 저장됩니다.

In [ ]:
# 베이스 모델 로드
print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)


베이스 모델을 업로드 합니다. return_dict=True는 모델의 출력을 딕셔너리 형태로 반환하도록 합니다.


In [ ]:
# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
peft_model = PeftModel.from_pretrained(base_model, adapter_path)
merged_model = peft_model.merge_and_unload()

NameError: name 'adapter_path' is not defined

PeftModel.from_pretrained() 함수는 베이스 모델 위에 학습된 LoRA 어댑터를 로드합니다. 이함수는 베이스 모델과 어댑터를 함께 관리하는 모델 객체를 생성합니다. merge_and_unload() 함수
는 LoRA 어댑터의 가중치를 베이스 모델에 병합합니다.

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

베이스 모델에서 토크나이저를 로드합니다. 토크나이저는 텍스트를 토큰으로 변환하고 다시 텍스트로  변환하는 역할을 합니다.

In [ ]:
# 저장
print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("모델과 토크나이저 저장 완료")

병합된 모델, 즉 학습된 최종 모델을 저장합니다. 마지막으로 모델과 같은 경로에 토크나이저를 저장합니다. 일반적으로 모델과 토크나이저는 함께저장합니다. 또한 허깅페이스에도 모델과 토크나이저를 같이 업로드할 것입니다. 이는 사용자들에게 해당 모델을 사용할때는 해당 토크나이저를 사용해야 한다는 것을 알려주는 역할을 합니다.

###2. 허깅페이스 업로드
병합된 모델을 허깅페이스 허브에 업로드 하여 다른 사람과 공유하거나 나중에 쉽게 사용할 수 있도록 할 수 있습니다.

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

username = "jin63"


먼저 허깅페이스 허브와 상호작용하기 위한 API 클라이언트를 초기화합니다. HfApi()는 허깅페이스 허브의 모든 기능에 접근할 수 있는 고수준 인터페이스를 제공합니다. 이 API 를 통해 저장소 생성, 파일 업로드, 모델 정보 조회 등의 작업을 수행할 수 있습니다. username 변수에는 자신의 허깅페이스 사용자명 (허깅페이스 로그인 후 보이는 닉네임) 을 설정합니다. 저자의 경우 “iamjoon” 이라는 사용자명을
사용하고 있습니다

In [ ]:
MODEL_NAME =  'qwen2-7b-rag-ko-checkpoint-285'


업로드할 모델의 이름을 정의합니다. 모델명은 가능한 한 구체적이고 설명적으로 작성하는 것이 좋습니다. 여기서는 qwen2-7b-rag-ko-checkpoint-285라는 이름으로 업로드합니다

In [ ]:
api.create_repo(
token="hf로 시작하는 여러분의 키 값",
repo_id=f"{username}/{MODEL_NAME}",
repo_type="model"
)

허깅페이스 허브에 새로운 저장소를 생성합니다. token 매개변수에는 허깅페이스 액세스 토큰을 입력 합니다. 이 토큰은 허깅페이스 웹사이트의 로그인 후에 발급받는 키 값입니다. Settings > Access
Tokens에서 생성할 수 있으며, Write 권한이 있는 토큰이어야 합니다. repo_id는 저장소의 고유 식별자로, 사용자명/모델명 형태로 구성됩니다. repo_type="model"은 이 저장소가 모델을 저장하는 용도임을 명시합니다.


In [ ]:
api.upload_folder(
    token="hf로 시작하는 여러분의 키값",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merged",
)


로컬에 저장된 모델 폴더를 허깅페이스 허브에 업로드합니다. folder_path="merged"는 앞서 병
합된 모델이 저장된 로컬 폴더 경로를 지정합니다. 이 폴더에는 모델 가중치 파일, 설정 파일, 프로세서
(VLM 의 토크나이저) 파일 등 모델 실행에 필요한 모든 파일들이 포함되어 있습니다. 업로드 과정에서는
폴더 내의 모든 파일이 허깅페이스 허브로 전송되며, 파일 크기에 따라 몇 분에서 몇십 분이 소요될 수 있
습니다.
업로드가 완료되면 다른 사용자들이 이 모델을 다운로드하고 사용할 수 있게 됩니다. 업로드된 모
델은 https://huggingface.co/{username}/{MODEL_NAME} 주소에서 확인할 수 있습니
다. 저자가 업로드 한 모델의 경우 https://huggingface.co/iamjoon/qwen2-7b-rag-kocheckpoint-285 주소에서 확인할 수 있습니다.